# 01: Reading Experiments

This notebook demonstrates the basics of reading experiment data using the Yanex Results API.

## What You'll Learn

- Importing and using the Results API
- Getting single and multiple experiments
- Filtering experiments by tags
- Exploring the `Experiment` object
- Reading parameters and metrics

## Prerequisites

**Before running this notebook**, you need to create sample experiments using the multi-step metrics CLI example with a parameter sweep.

Run this command from the `examples/cli/05_multi_step_metrics/` directory:

```bash
cd examples/cli/05_multi_step_metrics
yanex run train_model.py \
  --param "epochs=10,20,30" \
  --param "learning_rate=logspace(-4, -1, 4)" \
  --param "batch_size=32" \
  --tag results-demo \
  --parallel 0
```

This creates **12 experiments** in parallel (3 epochs × 4 learning rates) with the tag `results-demo`. We'll use this tag to filter and analyze these specific experiments throughout this notebook.

**What this trains:**
- A simple neural network training simulation
- Logs metrics at each epoch: `train_loss`, `train_accuracy`
- Occasionally logs validation metrics: `val_loss`, `val_accuracy`
- Parameters: `epochs`, `learning_rate`, `batch_size`

## Import Results API

The Results API is available under `yanex.results`. By convention, we import it as `yr`.

In [ ]:
import yanex.results as yr

## Getting Our Training Experiments

Use `get_experiments()` with the `tags` filter to retrieve our 12 training experiments.

In [ ]:
# Get our 12 experiments using the tag
experiments = yr.get_experiments(tags=["results-demo"])

print(f"Retrieved {len(experiments)} experiments")
print("\nExperiment IDs:")
for exp in experiments[:5]:
    print(f"  {exp.id}: {exp.status}")
if len(experiments) > 5:
    print(f"  ... and {len(experiments) - 5} more")

## Getting a Single Experiment: `get_experiment(id)`

When you know the experiment ID, use `get_experiment()` to retrieve it directly.

In [ ]:
# Pick the first experiment as an example
experiment_id = experiments[0].id
exp = yr.get_experiment(experiment_id)

print(f"Retrieved experiment: {exp.id}")
print(f"Status: {exp.status}")
print(f"Tags: {', '.join(exp.tags)}")

## The Experiment Object

The `Experiment` object provides access to all experiment data:

### Metadata Properties
- `id` - Unique 8-character hex ID
- `name` - Human-readable name
- `description` - Experiment description
- `status` - Status (completed, failed, cancelled, running)
- `tags` - List of tags
- `started_at` - Start timestamp (datetime)
- `completed_at` - End timestamp (datetime)
- `duration` - Duration (timedelta)
- `script_path` - Path to script
- `experiment_dir` - Path to experiment directory

### Data Access Methods
- `get_params()` - Get all parameters as dict
- `get_param(name)` - Get single parameter value
- `get_metrics()` - Get all metrics (list of dicts with timestamps)
- `get_metric(name)` - Get specific metric value(s)
- `get_artifacts()` - Get list of artifact paths

In [ ]:
# Explore an experiment in detail
exp = experiments[0]

print("=" * 60)
print(f"Experiment: {exp.id}")
print("=" * 60)

# Metadata
print(f"\nStatus: {exp.status}")
print(f"Tags: {', '.join(exp.tags)}")

# Timing
print(f"\nStarted: {exp.started_at}")
print(f"Completed: {exp.completed_at}")
print(f"Duration: {exp.duration}")

# Paths
print(f"\nScript: {exp.script_path}")
print(f"Directory: {exp.experiment_dir}")

## Reading Parameters

Parameters are the configuration values used to run the experiment. Our training experiments have three parameters: `epochs`, `learning_rate`, and `batch_size`.

In [ ]:
# Get all parameters as a dictionary
params = exp.get_params()

print("All parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

# Get specific parameters
epochs = exp.get_param("epochs")
learning_rate = exp.get_param("learning_rate")
batch_size = exp.get_param("batch_size")

print("\nTraining configuration:")
print(f"  Epochs: {epochs}")
print(f"  Learning rate: {learning_rate}")
print(f"  Batch size: {batch_size}")

## Reading Metrics

Metrics are the results logged during experiment execution. Our training script logs:
- `train_loss` - Training loss at each epoch
- `train_accuracy` - Training accuracy at each epoch  
- `val_loss` - Validation loss (logged occasionally)
- `val_accuracy` - Validation accuracy (logged occasionally)

**Two ways to access metrics:**
1. `get_metrics()` - Returns list of metric dictionaries (includes timestamps, steps)
2. `get_metric(name)` - Returns value(s) for a specific metric

In [ ]:
# Get all metrics (full detail with timestamps and steps)
all_metrics = exp.get_metrics()

print(f"Total metric entries logged: {len(all_metrics)}")
print("\nFirst 3 metric entries:")

for metric in all_metrics[:3]:
    print(f"\n  Step {metric['step']} @ {metric['timestamp']}:")
    for key, value in metric.items():
        if key not in ["step", "timestamp", "last_updated"]:
            print(f"    {key}: {value:.4f}")

### Accessing Specific Metrics

Use `get_metric()` to extract specific metrics across all steps.

In [ ]:
# Get training loss across all epochs
train_loss = exp.get_metric("train_loss")
train_accuracy = exp.get_metric("train_accuracy")

print(f"Training progression ({len(train_loss)} epochs):")
print(f"\nTrain Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")

# Show final values
print("\nFinal metrics:")
print(f"  Train Loss: {train_loss[-1]:.4f}")
print(f"  Train Accuracy: {train_accuracy[-1]:.4f}")

## Comparing Multiple Experiments

Let's look at the parameter configurations and final performance across all 12 experiments.

In [ ]:
print("All 12 Training Experiments")
print("=" * 90)
print(
    f"{'ID':<12} {'Epochs':<8} {'Learn Rate':<12} {'Final Loss':<12} {'Final Acc':<12}"
)
print("-" * 90)

for exp in experiments:
    epochs = exp.get_param("epochs")
    lr = exp.get_param("learning_rate")

    # Get final metrics
    train_loss = exp.get_metric("train_loss")
    train_acc = exp.get_metric("train_accuracy")

    final_loss = train_loss[-1]
    final_acc = train_acc[-1]

    print(
        f"{exp.id:<12} {epochs:<8} {lr:<12.6f} {final_loss:<12.4f} {final_acc:<12.4f}"
    )

## Finding the Best Experiment

Let's find which hyperparameter combination gave the best final training accuracy.

In [ ]:
# Use get_best() to find the experiment with highest train_accuracy
best_exp = yr.get_best("train_accuracy", maximize=True, tags=["results-demo"])

print("Best Experiment:")
print("=" * 60)
print(f"ID: {best_exp.id}")
print("Hyperparameters:")
print(f"  Epochs: {best_exp.get_param('epochs')}")
print(f"  Learning rate: {best_exp.get_param('learning_rate'):.6f}")
print(f"  Batch size: {best_exp.get_param('batch_size')}")
print("Final Performance:")
print(f"  Train Loss: {best_exp.get_metric('train_loss')[-1]:.4f}")
print(f"  Train Accuracy: {best_exp.get_metric('train_accuracy')[-1]:.4f}")
print(f"Duration: {best_exp.duration}")

## Filtering by Status

You can filter experiments by various criteria to focus your analysis.

In [ ]:
# Filter our experiments by status
completed = yr.get_experiments(tags=["results-demo"], status="completed")
failed = yr.get_experiments(tags=["results-demo"], status="failed")

print("Status breakdown:")
print(f"  Completed: {len(completed)}/12")
print(f"  Failed: {len(failed)}/12")

if failed:
    print("\nFailed experiments:")
    for exp in failed:
        print(
            f"  {exp.id}: {exp.get_param('epochs')} epochs, lr={exp.get_param('learning_rate'):.6f}"
        )

## Key Takeaways

✅ **Results API Basics:**
- `import yanex.results as yr` - Import convention
- `yr.get_experiments(tags=[...])` - Query experiments by tag
- `yr.get_experiment(id)` - Get single experiment by ID
- Filter with `status`, `tags`, `name`, etc.

✅ **Experiment Object:**
- Metadata properties: `id`, `status`, `tags`, `started_at`, `duration`, etc.
- Parameter access: `get_params()`, `get_param(name)`
- Metric access: `get_metrics()` (with timestamps), `get_metric(name)` (values only)

✅ **Multi-Step Metrics:**
- `get_metric(name)` returns a list of values (one per step)
- Access final value with `[-1]`
- `get_metrics()` includes full step and timestamp information

✅ **Use Cases:**
- Reading individual experiment results
- Extracting specific parameters or metrics
- Finding best experiments manually

## Next Steps

Continue to **Notebook 02: Filtering and Comparing** to learn:
- Using `compare()` to create pandas DataFrames
- Analyzing multiple experiments efficiently
- Advanced filtering patterns
- Finding optimal hyperparameters with pandas